In [94]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict
import datetime
from tqdm import tqdm
import spacy


data_path = '/Users/ruoyangzhang/Documents/PythonWorkingDirectory/news_exploration/Data/'

ValueError: unknown locale: UTF-8

### Importing data

In [47]:
data_1 = pd.read_csv(data_path + 'all-the-news/articles1.csv')

In [48]:
data_1 = data_1.drop(['Unnamed: 0'], axis = 1)

In [49]:
data_1.head()

,id,title,publication,author,date,year,month,url,content
0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


### Data repartition

- remove punctuations
- lower case everything
- reduce consecutive white spaces
- tokenisation



we repartition the data into intervals of 7 days with a padding period of 23 days:

- this means that for one period, we have 30 days of articles
- the padding is there to ensure the smooth transition between periods
- as well as to ensure we have just slightly more volume, since it is recommended to apply W2V on sufficiently large datasets

In [ ]:
# we borrow a few functions from fastai's NLP course on document classif https://github.com/fastai/fastai/blob/master/courses/dl2/imdb.ipynb

re1 = re.compile(r'  +')

def fixup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))



In [50]:
data_1.date = [datetime.datetime.strptime(day, '%Y-%m-%d') for day in data_1.date]

In [80]:
min_date = min(data_1.date)
max_date = max(data_1.date)

In [81]:
min_date, max_date

(Timestamp('2016-01-01 00:00:00'), Timestamp('2016-12-31 00:00:00'))

In [82]:
data_1.head()

,id,title,publication,author,date,year,month,url,content
0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,17291,"First, a Mixtape. Then a Romance. - The New Yo...",New York Times,Katherine Rosman,2016-12-31,2016.0,12.0,NaN,"Just how is Hillary Kerr, the founder of ..."
2,17292,Calling on Angels While Enduring the Trials of...,New York Times,Andy Newman,2016-12-31,2016.0,12.0,NaN,Angels are everywhere in the Muñiz family’s ap...
3,20113,"Clinton, Sanders and the Crowd Bring the Noise...",New York Times,James Poniewozik,2016-04-16,2016.0,4.0,NaN,"Thursday night, Democrats did what party peopl..."
4,20114,Yahoo’s Suitors Uncover Few Financial Details ...,New York Times,Vindu Goel and Michael J. de la Merced,2016-04-18,2016.0,4.0,NaN,SAN FRANCISCO — As Yahoo prepares to accept...


We filter the data to only focus on data from 2016

In [83]:
data_1 = data_1[[str(day).split('-')[0] == '2016' for day in data_1.date]].reset_index(drop = True)

In [84]:
data_1.date[0] + datetime.timedelta(days = 7)

Timestamp('2017-01-07 00:00:00')

In [89]:
# we could parallelise this process if the datasets are much bigger
begin_date = min_date
repartitioned_articles = []
for i in tqdm(range(int((int((max_date - min_date).days) - 30) / 7))):
    articles = data_1[[begin_date <= day < begin_date + datetime.timedelta(days = 30) for day in data_1.date]].content
    #print(len(articles))
    begin_date += datetime.timedelta(days = 7)
    repartitioned_articles.append(articles)

100%|██████████| 47/47 [00:07<00:00,  6.04it/s]


In [90]:
repartitioned_articles[0]

3693     Twitter is breaking down. In a weekend exodus,...
3695     This week on “Fox News Sunday,” Republican pre...
3705     Director Michael Bay’s riveting, heartbreaking...
3719     Philip Rucker and Robert Costa write in the Wa...
3732       actor Dustin Hoffman weighed in on the renew...
3734     In the name of racial diversity, another major...
3735     A former marketing executive for textbook publ...
3741     The White House released a   Jan. 4 which prev...
3759     A former McCain senior adviser blasted former ...
3766     OUAGADOUGOU, Burkina Faso (AP)  —     militant...
3779     Police in New York City are seeking the identi...
3820     The latest polls show GOP frontrunner Donald T...
3822     Fox News has uncovered a video from 2013 that ...
3831     As President Obama delivers a defensive State ...
3844     Last Wednesday, Elton John interrupted his per...
3845     Going forward, every piece about Republican fr...
3851     This article was written by Daniel John Sobies.

In [91]:
nlp = spacy.load('en')

NameError: name 'spacy' is not defined